In [3]:
import os
import sys

from PIL import Image, ImageDraw, ImageFont

In [1]:
class ConfigMapRender:
    
    def __init__(self, options):

        self.square = 45
        
        #colors
        self.config_black_color_rgb = (125, 144, 173)
        self.config_white_color_rgb = (187, 193, 211)
        self.config_neutral_color_rgb = (220, 220, 220)

        self.config_lastMove_color_rgb = (127, 255, 0)
        
        self.config_red_color_rgb = (255, 102, 0)
        
        #Font size
        self.config_font_size = square
        
        self.decoratorFont = ImageFont.truetype("FreeSerif-4aeK.ttf", 12)
        
        self.cols = ['a','b','c','d','e','f','g','h']
        
        if 'newspaper' in options:
            self.mode = 'newspaper' if options['newspaper'] else 'color'
        else:
            self.mode = 'color'
        
    def getConfig(self ):
        return self
        


class MapRender:
    
    def __init__(self, options):
        
        self.config = ConfigMapRender(options)
        
        self.pieces = {
            'r':{'letter':'r', 'unicode':u"\u265C"},
            'n':{'letter':'n', 'unicode':u"\u265E"},
            'b':{'letter':'b', 'unicode':u"\u265D"},
            'q':{'letter':'q', 'unicode':u"\u265B"},
            'k':{'letter':'k', 'unicode':u"\u265A"},
            'p':{'letter':'p', 'unicode':u"\u265F"},

            'R':{'letter':'R', 'unicode':u"\u2656"},
            'N':{'letter':'N', 'unicode':u"\u2658"},
            'B':{'letter':'B', 'unicode':u"\u2657"},
            'Q':{'letter':'Q', 'unicode':u"\u2655"},
            'K':{'letter':'K', 'unicode':u"\u2654"},
            'P':{'letter':'P', 'unicode':u"\u2659"},
        }
        

        # https://python-chess.readthedocs.io/en/latest/core.html#squares
        cellCursor=0
        _squaresNum = {}
        
        for n in range(1,9):
            for letter in self.config.cols:
                _squaresNum[ f'{letter}{n}' ] = cellCursor
                cellCursor += 1
        self.squaresNum = _squaresNum
        
        
        self.squares = [
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1],
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1],
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1],
          [1,0,1,0,1,0,1,0],
          [0,1,0,1,0,1,0,1]
        ]
        
    def drawDecorators( self, img ):
        
        square = self.config.square
        boardDim = 8*square
        
        # Board decorators: letter columns
        for idCol, col in enumerate(self.config.cols):
            imgC = ImageDraw.Draw(img) 
            imgC.text(
                  [(idCol*square)+ (square/2), boardDim + 2], 
                  col,
                  anchor="mm",
                  fill="black",
                  font= self.config.decoratorFont
                )
            
        # Board decorators: number rows
        for idRow in range(0,8):
            imgR = ImageDraw.Draw(img) 
            imgR.text(
                  [boardDim + 4, (idRow*square)+ (square/2) ], 
                  str(8-idRow),
                  anchor="mm",
                  fill="black",
                  font= self.config.decoratorFont
                )
        

    def draw( self, map, options ): # chess pieces_map
        
        square = self.config.square
        mode= self.config.mode
        
        boardDim = 8* square
        imgDim = boardDim + 20

        
        
        bgColor = (255,255,255) if mode=='newspaper' else self.config.config_neutral_color_rgb
        
        img = Image.new(
          mode="RGB",
          size=(imgDim, imgDim ),
          color= bgColor
          )

        # pixels = img.load()
        
        squareCursor = 63
        
        self.drawDecorators(img)
                
        
        if mode=='newspaper':
            dif = 5
            draw = ImageDraw.Draw(img)
            for xl in range( int(boardDim / dif)):
                inc = xl * dif
                draw.line(( 0, inc, inc, 0), fill = (44,44,44))
                draw.line((boardDim, inc, inc, boardDim), fill = (44,44,44))
            
        
        squareFromIdx = self.squaresNum[ options['lastMoveFromTo'][0] ]  ##  a1=>0
        squareToIdx = self.squaresNum[ options['lastMoveFromTo'][1] ] ## a1=>0

        unicode_font = ImageFont.truetype("FreeSerif-4aeK.ttf", config_font_size)
        for idR, rows in enumerate(squares):
            for idC, cell in enumerate(rows):
                img1 = ImageDraw.Draw(img) 
                shape = [
                  (idC*square),
                  (idR*square),
                  (idC*square) + square,
                  (idR*square) + square
                ]
                
                colorSquare =  config_white_color_rgb if cell == 1 else config_black_color_rgb
                
                # Last move paint squares
                if squareCursor == squareFromIdx or squareCursor == squareToIdx:
                    colorSquare = config_red_color_rgb
                  
                # In newspaper mode print just white squares
                if mode=='newspaper':
                    colorSquare = (255,255,255)
                    if cell == 1:
                        img1.rectangle(shape, fill =colorSquare)
                    if squareCursor == squareFromIdx or squareCursor == squareToIdx:
                        img1.ellipse(shape, outline =(0,0,0), width=1)
                    
                else:
                    img1.rectangle(shape, fill =colorSquare)


                if( squareCursor in map ):
                    letter = str(map[squareCursor])
                    pieceText = pieces[letter]['unicode']
                else:
                    pieceText = ''

                img1.text(
                  [(idC*square)+6, (idR*square)-4 ], 
                  pieceText,
                  anchor="mm",
                  fill="black",
                  font=unicode_font
                )
                squareCursor -= 1

        return img
        
    

In [6]:
# mapRender = MapRender()

### Usage in notebook

```
%run ../shared/util_mapRender.ipynb
```




```
utilsMapRender = MapRender()


Moves = utilsChess.createNodeFlagList(game)
map = Moves[5].board().piece_map()
img = utilsMapRender.draw(map)

img.save( os.path.join( productPath, filename ) , "JPEG", quality=100)
```